In [62]:
import os.path 

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import pickle 

import base64 
import email 
from bs4 import BeautifulSoup
import csv

import gspread
from google.oauth2.service_account import Credentials

import os
from datetime import datetime
from datetime import datetime, timedelta

In [2]:
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]#"https://www.googleapis.com/auth/gmail.send",'https://www.googleapis.com/auth/gmail.readonly','https://mail.google.com/', "https://www.googleapis.com/auth/gmail.metadata", "https://www.googleapis.com/auth/gmail.modify"]
client_secret_file = './credenciales/client_secret_python_gmail.json'

## conexion

In [3]:
creds = None
  
# The file token.pickle contains the user access token. 
# Check if it exists 
if os.path.exists('token.pickle'): 
  
    # Read the token from the file and store it in the variable creds 
    with open('token.pickle', 'rb') as token: 
        creds = pickle.load(token) 
  
# If credentials are not available or are invalid, ask the user to log in. 
if not creds or not creds.valid: 
    if creds and creds.expired and creds.refresh_token: 
        creds.refresh(Request()) 
    else: 
        flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES) 
        creds = flow.run_local_server(port=0) 
  
    # Save the access token in token.pickle file for the next run 
    with open('token.pickle', 'wb') as token: 
        pickle.dump(creds, token) 
  
# Connect to the Gmail API 
service = build('gmail', 'v1', credentials=creds) 

## requests

#### Ver las etiquetas (LABELS)

In [4]:
results_labels = service.users().labels().list(userId="me").execute().get("labels")
results_labels[0]

{'id': 'CHAT',
 'name': 'CHAT',
 'messageListVisibility': 'hide',
 'labelListVisibility': 'labelHide',
 'type': 'system'}

#### TODO EL LISTADO

In [5]:
# request a list of all the messages 
emailIdList = service.users().messages().list(userId='me').execute().get('messages')
emailIdList[0]

{'id': '18c1ddad2928b477', 'threadId': '18c1ddad2928b477'}

In [6]:
for email_ids in emailIdList: 
    # Get the message from its id 
    email_parts = service.users().messages().get(userId='me', id=email_ids['id']).execute()

In [7]:
email_parts["labelIds"]

['IMPORTANT', 'CATEGORY_UPDATES', 'INBOX']

#### POR TITULO

In [8]:
emailIdList_filter_by_title = service.users().messages().list(userId='me', q=f'subject:test gmail csv').execute().get('messages')

In [9]:
for email_ids in emailIdList_filter_by_title: 
    # Get the message from its id 
    email_parts = service.users().messages().get(userId='me', id=email_ids['id']).execute()

In [10]:
email_ids

{'id': '18c1ae8bcff0d28c', 'threadId': '18c1ae886f268167'}

### obtener campos del email

##### Queries para poner dentro del method list (como en el siguiente ejemplo de subject):
  - subject:
  - from:
  - to:
  - cc:
  - bcc:
  - label:
  - has:attachment
  - has:drive
  - has:document
  - has:spreadsheet
  - has:presentation
  - filename:
  - newer_than:2d
  - deliveredto:
  - OR or { }: --> from:amy OR from:david

https://support.google.com/mail/answer/7190?hl=en    --> All the terms and REGEXtation

In [11]:
emailIdList_filter_by_title = service.users().messages().list(userId='me', q=f'subject:test gmail csv').execute().get('messages')

In [12]:
for email_ids in emailIdList_filter_by_title: 
    # Get the message from its id 
    email_parts = service.users().messages().get(userId='me', id=email_ids['id']).execute()
    email_field_from = [name['value'] for name in email_parts['payload']['headers'] if name['name'] == 'From']
    email_field_to = [name['value'] for name in email_parts['payload']['headers'] if name['name'] == 'To']
    email_field_subject = [name['value'] for name in email_parts['payload']['headers'] if name['name'] == 'Subject']
    email_field_date = [name['value'] for name in email_parts['payload']['headers'] if name['name'] == 'Date'] 

In [13]:
email_field_date

['Wed, 29 Nov 2023 12:47:52 +0100']

In [14]:
email_field_from

['Alv Sz <alvarosaezsanchez@gmail.com>']